# Exercise 2
Today we are going to continue work on pointclouds.
We will work on trying to cluster pointclouds to be able to segment them.
    
If you do not have sklearn installed make sure to **pip install scikit-learn**

In [1]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans, k_means

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def draw_labels_on_model(pcl,labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    print("%s has %d clusters" % (pcl_name, max_label + 1))
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])



## On a cube.
We createa a point cloud using open3d.
Our goal is to segment each side using k means.

In [3]:
pcl_name = 'Cube'
density = 1e4 # density of sample points to create
pcl = o3d.geometry.TriangleMesh.create_box().sample_points_uniformly(int(density))
eps = 0.4
print("%s has %d points" % (pcl_name, np.asarray(pcl.points).shape[0]))
o3d.visualization.draw_geometries([pcl])

Cube has 10000 points


If we just use Kmeans out of the box with the pointcloud we will get the following


Note that pressing plus and minus in the viewer will increase/decrease the size of the points in the viewer

In [4]:
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
xyz = np.asarray(pcl.points)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 


## Exercises

### A) K means continued.

Combine the point cloud points (xyz) with the normals and do k-means.
As we can see we got 6 clusters but they do not span a side.

We try again but this time we instead use the normals of the cube.
The normals for each plane should be parralell with the other normals from said plane.

**NB** Using only normals wont work either, as two and two sides have parallell normals. Combine with the xyzs

In [5]:
#############
## Normals ##
#############

pcl.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(pcl.normals)

In [6]:
#############
## Combine ##
#############

xyz_n = np.hstack((xyz, normals))
xyz_n.shape

(10000, 6)

In [7]:
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


### B) 
Try weighting either the points or normals by scaling them by some factor, can segment each of the faces of the cube?

In [8]:
xyz_n_scaled = np.hstack((xyz*2, normals))
labels = km.fit_predict(xyz_n_scaled)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


### C)
Try to cluster all the different shapes using k means.


In [9]:
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=2).translate(
    (d, -d, 0))
mesh = mesh.sample_points_uniformly(int(1e5), 0.5)
o3d.visualization.draw_geometries([mesh])

In [30]:
km2 = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
mesh_xyz = np.asarray(mesh.points)
labels = km2.fit_predict(mesh_xyz)
draw_labels_on_model(mesh, labels)

Cube has 6 clusters


In [31]:
labels

array([1, 1, 1, ..., 4, 4, 4])


### D)
Now try with the pointcloud in "pointclouds/fragment.ply"
Are you able to cluster the pointcloud?

What features here would it make sense to cluster?
- fpfh features?
- xyz
- normals 
- colors

Are you able to get clusters that make sense? Why?


In [11]:
pcl2 = o3d.io.read_point_cloud("TestData/fragment.ply")
o3d.visualization.draw_geometries([pcl2])

In [12]:

# Get the points from the pointcloud as nparray
frag_xyz = np.asarray(pcl2.points)
frag_c = np.asarray(pcl2.colors)
frag_xyz_c = np.hstack((frag_xyz, frag_c))

# Estimate and extract normal vectors
pcl2.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals_2 = np.asarray(pcl2.normals)
frag_xyz_n = np.hstack((frag_xyz, normals_2))
frag_xyz_n_c = np.hstack((frag_xyz_n, frag_c))

In [13]:
#frag_hsv = plt.colors.rgb_to_hsv(frag_c)
def rgb_to_hsv(rgb):
    """
    >>> from colorsys import rgb_to_hsv as rgb_to_hsv_single
    >>> 'h={:.2f} s={:.2f} v={:.2f}'.format(*rgb_to_hsv_single(50, 120, 239))
    'h=0.60 s=0.79 v=239.00'
    >>> 'h={:.2f} s={:.2f} v={:.2f}'.format(*rgb_to_hsv_single(163, 200, 130))
    'h=0.25 s=0.35 v=200.00'
    >>> np.set_printoptions(2)
    >>> rgb_to_hsv(np.array([[[50, 120, 239], [163, 200, 130]]]))
    array([[[   0.6 ,    0.79,  239.  ],
            [   0.25,    0.35,  200.  ]]])
    >>> 'h={:.2f} s={:.2f} v={:.2f}'.format(*rgb_to_hsv_single(100, 100, 100))
    'h=0.00 s=0.00 v=100.00'
    >>> rgb_to_hsv(np.array([[50, 120, 239], [100, 100, 100]]))
    array([[   0.6 ,    0.79,  239.  ],
           [   0.  ,    0.  ,  100.  ]])
    """
    input_shape = rgb.shape
    rgb = rgb.reshape(-1, 3)
    r, g, b = rgb[:, 0], rgb[:, 1], rgb[:, 2]

    maxc = np.maximum(np.maximum(r, g), b)
    minc = np.minimum(np.minimum(r, g), b)
    v = maxc

    deltac = maxc - minc
    s = deltac / maxc
    deltac[deltac == 0] = 1  # to not divide by zero (those results in any way would be overridden in next lines)
    rc = (maxc - r) / deltac
    gc = (maxc - g) / deltac
    bc = (maxc - b) / deltac

    h = 4.0 + gc - rc
    h[g == maxc] = 2.0 + rc[g == maxc] - bc[g == maxc]
    h[r == maxc] = bc[r == maxc] - gc[r == maxc]
    h[minc == maxc] = 0.0

    h = (h / 6.0) % 1.0
    res = np.dstack([h, s, v])
    return res.reshape(input_shape)
frag_hsv = rgb_to_hsv(frag_c)

C:\Users\henri\AppData\Local\Temp\ipykernel_20244\3044577194.py:28: RuntimeWarning: invalid value encountered in true_divide
  s = deltac / maxc


In [14]:
frag_hsv[:,0].reshape(len(frag_hsv[:,0]), 1)

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.05555556],
       [0.0530303 ],
       [0.0530303 ]])

In [15]:
################
# Only normals #
################
km3 = KMeans(n_clusters=3, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(normals_2)
draw_labels_on_model(pcl2, labels)

Cube has 3 clusters


In [16]:
############
# Only xyz #
############
km3 = KMeans(n_clusters=10, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_xyz)
draw_labels_on_model(pcl2, labels)

Cube has 10 clusters


In [17]:
############
# Only RGB #
############
km3 = KMeans(n_clusters=3, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_c)
draw_labels_on_model(pcl2, labels)

Cube has 3 clusters


In [18]:
############
# Only HSV #
############
frag_h = frag_hsv[:,0]
frag_s = frag_hsv[:,1] # Contains nans :I
frag_v = frag_hsv[:,2]

km3 = KMeans(n_clusters=2, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_v.reshape(len(frag_v), 1))
draw_labels_on_model(pcl2, labels)

Cube has 2 clusters


In [19]:
#################
# Normals + xyz #
#################
km3 = KMeans(n_clusters=10, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_xyz_n)
draw_labels_on_model(pcl2, labels)

Cube has 10 clusters


In [20]:
############
# All data #
############
frag_all = np.hstack((frag_xyz, frag_c, normals_2, frag_v.reshape(len(frag_v), 1)))
km3 = KMeans(n_clusters=12, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_all)
draw_labels_on_model(pcl2, labels)

Cube has 12 clusters


In [21]:
###############
# V + normals #
###############
frag_v_n = np.hstack((normals_2, frag_v.reshape(len(frag_v), 1)))
km3 = KMeans(n_clusters=12, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_v_n)
draw_labels_on_model(pcl2, labels)

Cube has 12 clusters


In [22]:
###############
# V + normals #
###############
frag_h_v = np.hstack((frag_h.reshape(len(frag_h), 1), frag_v.reshape(len(frag_v), 1)))
km3 = KMeans(n_clusters=12, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_h_v)
draw_labels_on_model(pcl2, labels)

Cube has 12 clusters


In [23]:
########################
# Stack wrong order :D #
########################
frag_depth = np.vstack((frag_xyz, frag_c, normals_2))
km3 = KMeans(n_clusters=12, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)
labels = km3.fit_predict(frag_depth)
draw_labels_on_model(pcl2, labels)

Cube has 12 clusters
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 



### E)
Use the built in cluster_dbscan algorithm.
Tweak the parameters and see what you get out.

Attempt on the combined figures and on "fragment.ply"

In [24]:
# We have 
#   - mesh: many figures
#   - pcl: one box
#   - pcl2: the livingroom

In [25]:
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.2

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
draw_labels_on_model(pcl, labels)

Cube has 1 clusters


In [26]:
eps = 0.2
min_points = 10

labels = np.array(mesh.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
draw_labels_on_model(mesh, labels)

Cube has 6 clusters


**Livingroom**
With DBScan if something is connected it will get clustered. Try to use other propterties than xyz and see what happens!

In [27]:
eps = 0.02
min_points = 10

labels = np.array(pcl2.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
draw_labels_on_model(pcl2, labels)

Cube has 10 clusters


In [28]:
from sklearn.cluster import DBSCAN

In [29]:
eps = 0.02
min_points = 10

clustering = DBSCAN(eps=eps, min_samples=min_points).fit(frag_xyz)
labels = clustering.labels_
draw_labels_on_model(pcl2, labels)

Cube has 10 clusters
